## **Introdução: Estrutura do nosso Fine tuning**



## **PARTE 1: CONFIGURAÇÃO INICIAL E CARREGAMENTO DO MODELO**


# 1.1 - Setup do Ambiente
- Montagem do Google Drive e instalação das dependências (Unsloth, PyTorch, etc.)

# 1.2 - Carregamento do Modelo Base
- Carregamento do Llama 3.1-8B com quantização 4-bit
- Configuração do tokenizador


## **PARTE 2: LIMPEZA E PROCESSAMENTO DOS DADOS**

### 2.1 - Pré-processamento dos Dados
- Limpeza, normalização e remoção de duplicatas dos datasets

### 2.2 - Formatação para Fine-tuning
- Conversão para formato instruction-input-output
- Processamento de 250k registros de treino e 2.5k de teste

## **PARTE 3: PREPARAÇÃO E TESTE INICIAL DO MODELO**

### 3.1 - Configuração LoRA
- Aplicação do método LoRA para fine-tuning eficiente

### 3.2 - Teste Baseline
- Inferência com modelo base (antes do treinamento customizado)

## **PARTE 4: TREINAMENTO COMPLETO DO MODELO**

### 4.1 - Configuração do Treinamento
- Setup do SFTTrainer com hiperparâmetros otimizados

### 4.2 - Execução do Fine-tuning
- Treinamento por 60 passos com monitoramento de métricas

### 4.3 - Salvamento do Modelo
- Armazenamento dos pesos LoRA e tokenizador treinados

## **PARTE 5: TESTE FINAL DO MODELO TREINADO**

### 5.1 - Inferência Pós-treinamento
- Teste com dados customizados usando modelo fine-tuned

### 5.2 - Análise dos Resultados
- Comparação entre modelo base vs modelo treinado

## **PARTE 6: Exemplo dos arquivos utilizados**

### 6.1 - trn.json
- Dataset de treinamento

### 6.2 - tst.json
- Dataset de testes

### 6.3 - formatted_train_dataset.json
- Dataset de treino formatado

### 6.4 - formatted_test_dataset.json
- Dataset de testes formatado


### **▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄**

## **PARTE 1: CONFIGURAÇÃO INICIAL E CARREGAMENTO DO MODELO**

# **Montagem do Google Drive**

**from google.colab import drive:** Importa a biblioteca necessária para montar o Google Drive no ambiente Colab.
**drive.mount('/content/drive'):** Monta o Google Drive no diretório /content/drive, permitindo que o notebook acesse arquivos armazenados no seu Drive.

In [ ]:
# Somente para uso do Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Instalação do Unsloth e Dependências**

**%%capture:** Suprime a saída desta célula.
**import os:** Importa a biblioteca os para interagir com o sistema operacional.

O código verifica se está em um ambiente Colab ("COLAB_" not in "".join(os.environ.keys())).

Se não estiver no Colab (o que não é o caso neste contexto), instala o Unsloth via pip.
**Se estiver no Colab:**
Instala as dependências essenciais:
 bitsandbytes,
 accelerate,
 xformers,
 peft,
 trl,
 triton,
 cut_cross_entropy,
 unsloth_zo
com pip install --no-deps. O no-deps evita conflitos de versão.
Instala outras dependências:
sentencepiece,
protobuf,
datasets,
huggingface_hub,
hf_transfer
 necessárias para manipulação de dados e interação com o Hugging Face Hub.
Instala o Unsloth via pip (neste caso, sem dependências diretas, assumindo que as anteriores cobrem o necessário).

**Nota:** A linha comentada !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git" é uma forma alternativa de instalar a versão mais recente do Unsloth diretamente do GitHub, focada no Colab.

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

# **Instalação de Xformers (Versão Específica)**

Importa __version__ do torch e Version do packaging.version para verificar a versão do PyTorch.
Define a versão do xformers a ser instalada com base na versão do PyTorch (xformers==0.0.27 para PyTorch < 2.4.0, xformers sem versão específica para versões mais recentes).
Instala o xformers (com a versão definida), trl, peft, accelerate, bitsandbytes e triton com pip install --no-deps. Esta célula parece redundante após a Célula 3 em um ambiente Colab, mas garante que a versão correta do xformers seja usada.

In [ ]:
# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

# **Importações e Configurações Adicionais**

Importa classes e funções das bibliotecas instaladas (SFTTrainer, TrainingArguments, is_bfloat16_supported, load_dataset, Dataset, pandas, json, yaml, unicodedata, re, os).
os.environ["WANDB_DISABLED"] = "true": Desabilita a integração com o Weights & Biases (WandB), uma ferramenta comum para monitoramento de treinamento, para evitar a solicitação de uma API Key.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from datasets import load_dataset, Dataset
import pandas as pd
import json, yaml
import unicodedata
import re

import os

# Desabilitar para não solicitar APIKey
os.environ["WANDB_DISABLED"] = "true"

<ipython-input-4-28e0616b7742>:3: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import is_bfloat16_supported


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


# **Carregamento do Modelo Base com Unsloth**

from unsloth import FastLanguageModel: Importa a classe principal do Unsloth para carregar modelos rapidamente.
import torch: Importa a biblioteca PyTorch.
Define max_seq_length (comprimento máximo da sequência de entrada), dtype (tipo de dado, None para detecção automática) e load_in_4bit (habilita quantização em 4 bits para reduzir o uso de memória VRAM).
Define uma lista fourbit_models com nomes de modelos otimizados para 4 bits pelo Unsloth.
model, tokenizer = FastLanguageModel.from_pretrained(...): Carrega o modelo pré-treinado unsloth/Meta-Llama-3.1-8B utilizando a função otimizada do Unsloth. Configura o comprimento máximo da sequência, tipo de dado e ativa a quantização em 4 bits.
Importa CHAT_TEMPLATES e get_chat_template para configurar o tokenizador com um template de chat específico.
tokenizer = get_chat_template(tokenizer, chat_template = "gemma-3", ...): Configura o tokenizador para usar o template de chat "gemma-3". Nota: O template "gemma-3" pode não ser o ideal para o modelo Llama 3.1. Seria mais apropriado usar um template compatível com Llama 3.1.

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

from unsloth.chat_templates import CHAT_TEMPLATES, get_chat_template
print(list(CHAT_TEMPLATES.keys()))

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

['unsloth', 'zephyr', 'chatml', 'mistral', 'llama', 'vicuna', 'vicuna_old', 'vicuna old', 'alpaca', 'gemma', 'gemma_chatml', 'gemma2', 'gemma2_chatml', 'llama-3', 'llama3', 'phi-3', 'phi-35', 'phi-3.5', 'llama-3.1', 'llama-31', 'llama-3.2', 'llama-3.3', 'llama-32', 'llama-33', 'qwen-2.5', 'qwen-25', 'qwen25', 'qwen2.5', 'phi-4', 'gemma-3', 'gemma3', 'qwen-3', 'qwen3']


### **▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄**

## **PARTE 2: LIMPEZA E PROCESSAMENTO DOS DADOS**

# **Função prepara_dataset**

Define a função prepara_dataset(inpfile, outfile, tam_regs=None) para pré-processar os datasets de treinamento e teste.
Carrega dados de um arquivo JSON de entrada (inpfile) linha por linha, tratando possíveis erros de decodificação JSON.
Filtra os dados para incluir apenas itens com 'title' e 'content'.
Converte os dados em um DataFrame pandas.
Define a função normalizar para remover caracteres de controle e excesso de espaços, normalizando o texto.
Aplica a função normalizar às colunas 'title' e 'content'.
Remove linhas onde 'title' ou 'content' estão vazios após a normalização.
Remove linhas duplicadas com base em 'title' e 'content'.
Reseta o índice do DataFrame.
Cria as colunas 'instruction', 'input' e 'output' no formato necessário para o fine-tuning, seguindo a estrutura de "instruction-input-output".
Se tam_regs for especificado, amostra um subconjunto do DataFrame com tam_regs registros (com um random_state para reprodutibilidade).
Formata os dados em uma lista de dicionários com as colunas 'instruction', 'input' e 'output'.
Salva os dados formatados em um arquivo JSON de saída (outfile).
Imprime informações sobre o número de registros processados em cada etapa.
Retorna o número final de registros processados.

In [ ]:
# Efetua as diversas limpezas e estabilizações do dataset
def prepara_dataset(inpfile, outfile, tam_regs=None):
    # Primeiro carrega o JSON como uma tupla no vetor data
    data = []
    with open(inpfile, 'r', encoding='utf-8') as file:
        for line in file: # para cada linha
            try:
                item = json.loads(line)
                if 'title' in item and 'content' in item:
                    title = item['title']
                    content = item['content']
                    if title or content is not None:
                        data.append({'title': title, 'content': content})  # só nos interessa titulo e conteudo
            except json.JSONDecodeError:
                pass # bypass de leituras erradas

    df = pd.DataFrame(data) # Usa o pandas para converter uma lista em dataFrame
    print(f"Registros iniciais carregados: {len(df)}")

    # Função para remover carac de controle e excesso de espaço para ganhar tokens
    def normalizar(text):
        if not isinstance(text, str):
            return ""
        text = ''.join(ch for ch in text if unicodedata.category(ch)[0] != 'C')
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    # Aplica a função de normalização nas colunas 'title' e 'content'
    df['title'] = df['title'].apply(normalizar)
    df['content'] = df['content'].apply(normalizar)

    df = df[(df['title'].str.strip() != '') & (df['content'].str.strip() != '')] # remove brancos e vazios
    print(f"Registros após remoção de vazios: {len(df)}")

    df.drop_duplicates(subset=['title', 'content'], inplace=True) # remove duplicados
    print(f"Registros após remoção de duplicados: {len(df)}")

    df.reset_index(drop=True, inplace=True) # como mexemos, precisa resetar o índice

    # Cria as colunas 'instruction', 'input' e 'output' para o formato necessário
    df['instruction'] = "Answer the question based on the product title."
    df['input'] = df['title'].apply(lambda x: f"What is '{x}'?")
    df['output'] = df['content']

    # Verificar se tam_regs é maior que o número de registros disponíveis
    if tam_regs is not None:
        if tam_regs > len(df):
            print(f"AVISO: tam_regs ({tam_regs}) é maior que o número de registros disponíveis ({len(df)}). Usando todos os registros.")
        else:
            df = df.sample(n=tam_regs, random_state=42).reset_index(drop=True)

    formatted_data = df[['instruction', 'input', 'output']].to_dict(orient='records') # Formata para dicionários

    with open(outfile, 'w', encoding='utf-8') as output_file:
        json.dump(formatted_data, output_file, ensure_ascii=False, indent=4) # Salva o Json para uso

    print(f"Total de registros processados: {len(df)}")
    return len(df)

# **Definição dos Caminhos dos Arquivos**

Define as variáveis com os caminhos dos arquivos JSON originais de treinamento (dataset_de_treino) e teste (dataset_de_teste) no Google Drive.
Define as variáveis com os caminhos de saída para os datasets processados de treinamento (dataset_formatado) e teste (dataset_de_teste_formatado).

In [ ]:
# Definindo os caminhos dos arquivos de entrada e saída
dataset_de_treino = "/content/drive/MyDrive/Fiap/trn.json"
dataset_formatado = "/content/drive/MyDrive/Fiap/formatted_train_dataset.json"

dataset_de_teste = "/content/drive/MyDrive/Fiap/tst.json"
dataset_de_teste_formatado = "/content/drive/MyDrive/Fiap/formatted_test_dataset.json"


# **Execução do Pré-processamento para Treinamento**

Chama a função prepara_dataset para processar o arquivo de treinamento (dataset_de_treino) e salvar o resultado em dataset_formatado. Limita o número de registros para 650 (tam_regs=650).

In [ ]:
prepara_dataset(dataset_de_treino, dataset_formatado, tam_regs=250000)


Registros iniciais carregados: 2248619
Registros após remoção de vazios: 1390403
Registros após remoção de duplicados: 1367081
Total de registros processados: 250000


250000

# **Execução do Pré-processamento para Teste**

Chama a função prepara_dataset para processar o arquivo de teste (dataset_de_teste) e salvar o resultado em dataset_de_teste_formatado. Limita o número de registros para 150 (tam_regs=150).

In [ ]:
prepara_dataset( dataset_de_teste, dataset_de_teste_formatado, tam_regs=2500)

Registros iniciais carregados: 970237
Registros após remoção de vazios: 599743
Registros após remoção de duplicados: 593737
Total de registros processados: 2500


2500

### **▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄**

## **PARTE 3: PREPARAÇÃO E TESTE INICIAL DO MODELO**

# **Configuração do Modelo para Fine-tuning (LoRA)**

model = FastLanguageModel.get_peft_model(...): Configura o modelo carregado anteriormente para fine-tuning utilizando o método LoRA (Low-Rank Adaptation).
Define o rank (r) das matrizes LoRA (16).
Define os módulos alvo (target_modules) do modelo onde as adaptações LoRA serão aplicadas (módulos relacionados à atenção e feed-forward).
Define lora_alpha (escala das atualizações LoRA) e lora_dropout.
Define bias como "none".
Habilita use_gradient_checkpointing = "unsloth" para otimizar o uso de memória VRAM.
Define random_state para reprodutibilidade.
Outras configurações (use_rslora, loftq_config) são mantidas como padrão.

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,                                                     # Rank das matrizes LoRA podendo ser 8, 16, 32, 64, 128. Quanto menor o valor, menos custo computacional e menos poder de ajuste.

    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",   # Módulos do modelo que serão adaptados
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,                                            # Controla a escala de atualização das matrizes LoRA
    lora_dropout = 0,                                           # Ajuda a prevenir overfitting em tarefas específicas, onde 0 é otimizado e valores como 0.1 ou 0.2 podem ser úteis para baixo volume de dados.
    bias = "none",                                              # Nenhuma definição necessária para manter a otimização

    use_gradient_checkpointing = "unsloth",                     # Utilização de 30% menos de VRAM e 2x mais rápido para grandes contextos
    random_state = 1961,                                        # Número randômico

    use_rslora = False,
    loftq_config = None,
)

Unsloth 2025.5.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# **Definição do Template Alpaca Prompt**

Define a string alpaca_prompt que representa a estrutura do template de prompt a ser usado para formatar os dados para o treinamento. Este template segue o formato "instruction-input-response".

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


# **Função formatting_prompts_func**

Define a função formatting_prompts_func que será usada para formatar o dataset no template do prompt.
Obtém as colunas 'instruction', 'input' e 'output' do dataset.
Itera sobre os exemplos e formata cada um utilizando o alpaca_prompt e adicionando o EOS_TOKEN (End-Of-Sentence Token) do tokenizador no final.
Retorna um dicionário com a coluna 'text' contendo os prompts formatados.

In [ ]:
EOS_TOKEN = tokenizer.eos_token            # Token que indica o fim de uma sequência
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Formata cada exemplo no template do prompt e adiciona o EOS_TOKEN.
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


# **Carregamento e Formatação de um Dataset de Exemplo (Yahma/Alpaca-cleaned)**

%%capture: Suprime a saída.
from datasets import load_dataset: Importa a função para carregar datasets do Hugging Face Hub.
dataset = load_dataset("yahma/alpaca-cleaned", split = "train"): Carrega um dataset de exemplo (yahma/alpaca-cleaned). Nota: Este dataset parece ser carregado apenas para demonstração ou teste inicial, pois o dataset customizado de produtos Amazon será carregado posteriormente.
dataset = dataset.map(formatting_prompts_func, batched = True,): Aplica a função formatting_prompts_func a este dataset de exemplo para formatá-lo.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

alpaca_data_cleaned.json:   0%|          | 0.00/44.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Map:   0%|          | 0/51760 [00:00<?, ? examples/s]

# **Habilitação da Inferência Rápida com Unsloth**

%%capture: Suprime a saída.
FastLanguageModel.for_inference(model): Otimiza o modelo para inferência, tornando-a até 2x mais rápida.

In [ ]:
%%capture
FastLanguageModel.for_inference(model) # Habilita inferência nativa que é 2x mais rápida

# **Exemplo de Inferência (Antes do Fine-tuning Completo)**

Prepara uma entrada (inputs) para o tokenizador utilizando o template alpaca_prompt com uma instrução e um input de exemplo ("Who was Ayrton Senna?"). O campo de resposta é deixado vazio para o modelo preencher.
from transformers import TextStreamer: Importa a classe TextStreamer para exibir a saída gerada pelo modelo em tempo real.
text_streamer = TextStreamer(tokenizer): Cria uma instância do TextStreamer.
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128): Gera texto utilizando o modelo carregado (que já tem a configuração LoRA, mas ainda não foi treinado com os dados customizados). A saída é transmitida pelo text_streamer.
print (text_streamer): Imprime o objeto TextStreamer (provavelmente para verificar seu conteúdo após a geração).

In [ ]:
# Prepara a entrada para o modelo fine-tuned.
inputs = tokenizer(
[
    alpaca_prompt.format(
       "Answer the question based on the title.",  # instruction em inglês
       "Who was Ayrton Senna?",                   # pergunta diferente do padrão treinado
       "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

print(text_streamer)


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the title.

### Input:
Who was Ayrton Senna?

### Response:
Ayrton Senna was a Brazilian Formula One racing driver who died in a car accident during a race in 1994.

### Explanation:
Ayrton Senna was a Brazilian Formula One racing driver who died in a car accident during a race in 1994.
<|end_of_text|>


### **▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄**

## **PARTE 4: TREINAMENTO COMPLETO DO MODELO**

# **Carregamento e Formatação do Dataset de Treinamento Customizado**

from datasets import load_dataset: Importa a função novamente (já importada na Célula 5).
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Fiap/formatted_train_dataset.json", split="train"): Carrega o dataset de treinamento pré-processado do arquivo JSON salvo na Célula 9.
dataset = dataset.map(formatting_prompts_func, batched = True,): Aplica a função formatting_prompts_func a este dataset customizado para formatá-lo para o treinamento.

In [ ]:
# Carrega o dataset formatado a partir do arquivo JSON.
dataset = load_dataset("json", data_files=dataset_formatado, split="train")
# Aplica a função de formatação de prompts ao dataset.
dataset = dataset.map(formatting_prompts_func, batched = True,)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/250000 [00:00<?, ? examples/s]

# **Configuração e Preparação do Treinador (SFTTrainer)**

Define o diretório de saída (modelos_treinado) para salvar os resultados do treinamento.
Cria uma instância do SFTTrainer, que é uma classe de conveniência para fine-tuning em tarefas de Sequence-to-Sequence.
Configura o trainer com o modelo (model), tokenizador (tokenizer), dataset de treinamento formatado (dataset), campo de texto no dataset (dataset_text_field), comprimento máximo da sequência (max_seq_length), número de processos para carregar dados (dataset_num_proc), e desabilita o packing (packing).
Configura os argumentos de treinamento (TrainingArguments):
per_device_train_batch_size: Tamanho do batch por dispositivo (GPU).
gradient_accumulation_steps: Número de passos para acumular gradientes antes de atualizar os pesos.
warmup_steps: Número de passos de aquecimento da taxa de aprendizado.
num_train_epochs: Número de épocas de treinamento (comentado, usando max_steps).
max_steps: Número máximo de passos de treinamento.
learning_rate: Taxa de aprendizado inicial.
fp16 e bf16: Configura o tipo de precisão (Float16 ou Bfloat16) com base na compatibilidade do hardware.
logging_steps: Frequência para registrar métricas.
optim: Otimizador a ser usado.
weight_decay: Penalidade L2.
lr_scheduler_type: Tipo de agendador de taxa de aprendizado.
seed: Semente para reprodutibilidade.
output_dir: Diretório para salvar os resultados.
report_to = "none": Desabilita relatórios para plataformas externas.
save_strategy="no": Não salva checkpoints intermediários.


In [ ]:
# Define o diretório onde os resultados do treinamento serão salvos.
modelos_treinado = "/content/drive/MyDrive/Fiap/Treinados"

from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported


trainer = SFTTrainer (
    model = model,                          # Modelo a ser treinado.
    tokenizer = tokenizer,                  # Tokenizador do modelo.
    train_dataset = dataset,                # Dataset de treinamento formatado.
    dataset_text_field = "text",            # Campo no dataset que contém o texto formatado.
    max_seq_length = max_seq_length,        # Comprimento máximo da sequência.
    dataset_num_proc = 2,                   # Número de processos para carregar dados.
    packing = False,                        # Desabilita packing (pode acelerar para sequências curtas).
    args = TrainingArguments (
        per_device_train_batch_size = 2,        # Tamanho do batch por GPU (reduzido para economizar memória).
        gradient_accumulation_steps = 4,        # Acumula gradientes por 2 passos antes de atualizar.
        warmup_steps = 5,                       # 5 passos de aquecimento da taxa de aprendizado.
        num_train_epochs = 2,                   # Número de épocas (desabilitado, usando max_steps).
        max_steps = 60,                         # Treina por no máximo 60 passos.
        learning_rate = 2e-4,                   # Taxa de aprendizado inicial.
        fp16 = not is_bfloat16_supported(),     # Usa FP16 se bfloat16 não for suportado.
        bf16 = is_bfloat16_supported(),         # Usa BF16 se suportado (mais estável).
        logging_steps = 1,                      # Registra métricas a cada passo.
        optim = "adamw_8bit",                   # Otimizador AdamW otimizado para 8 bits.
        weight_decay = 0.01,                    # Penalidade L2.
        lr_scheduler_type = "linear",           # Agendador de taxa de aprendizado linear.
        seed = 1961,                            # Semente para reprodutibilidade.
        output_dir = modelos_treinado,          # Diretório de saída.
        report_to = "none",                     # Não envia relatórios para plataformas externas.
        save_strategy="no",                     # Não salva checkpoints intermediários.
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/250000 [00:00<?, ? examples/s]

# **Exibição das Estatísticas de Memória da GPU (Antes do Treinamento)**

Obtém as propriedades da GPU.
Calcula e imprime a memória da GPU reservada e a memória total.

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
7.369 GB of memory reserved.


# **Execução do Treinamento**

trainer_stats = trainer.train(): Inicia o processo de treinamento utilizando as configurações definidas na Célula 18.

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 250,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.960600
2,2.534900
3,2.853500
4,2.840100
5,2.514400
6,2.550100
7,2.370400
8,2.195300
9,2.003800
10,1.775000


# **Exibição das Estatísticas de Memória e Tempo (Após o Treinamento)**

Calcula e imprime o tempo total de treinamento, a memória da GPU utilizada (pico reservado), a memória utilizada especificamente para o LoRA, e as porcentagens de uso em relação à memória total da GPU.


In [ ]:
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

460.476 seconds used for training.
7.67 minutes used for training.
Peak reserved memory = 8.154 GB.
Peak reserved memory for training = 0.785 GB.
Peak reserved memory % of max memory = 55.315 %.
Peak reserved memory for training % of max memory = 5.325 %.


# **Salvamento do Modelo e Tokenizador Fine-tuned**

model.save_pretrained(...): Salva os pesos LoRA treinados no diretório especificado.
tokenizer.save_pretrained(...): Salva o tokenizador (incluindo configurações de chat) no mesmo diretório.

In [ ]:
model.save_pretrained(modelos_treinado)
tokenizer.save_pretrained(modelos_treinado)

('/content/drive/MyDrive/Fiap/Treinados/tokenizer_config.json',
 '/content/drive/MyDrive/Fiap/Treinados/special_tokens_map.json',
 '/content/drive/MyDrive/Fiap/Treinados/tokenizer.json')

### **▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄**

## **PARTE 5: TESTE FINAL DO MODELO TREINADO**

# **Carregamento Opcional do Modelo Salvo**

Este bloco de código está comentado, mas demonstra como carregar o modelo e tokenizador salvos para inferência após o treinamento.
FastLanguageModel.from_pretrained(...): Carrega o modelo a partir do diretório salvo.
FastLanguageModel.for_inference(model): Otimiza o modelo carregado para inferência.

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = modelos_treinado, # path dos modelos treinados
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# **Exemplo de Inferência (Após o Fine-tuning)**

Prepara uma nova entrada para o tokenizador com uma instrução e um input diferente ("Rightly Dividing the Word") utilizando o alpaca_prompt. O campo de resposta é deixado vazio.
Cria uma nova instância de TextStreamer.
Gera texto utilizando o modelo que foi treinado com os dados customizados. A saída é transmitida pelo text_streamer.
Imprime o objeto TextStreamer.
Fluxo Geral do Notebook:

Configura o ambiente Colab (monta Drive, instala bibliotecas).
Carrega um modelo de linguagem base pré-treinado usando Unsloth (com quantização 4-bit).
Define uma função para limpar e formatar dados de produtos Amazon.
Processa os datasets de treinamento e teste customizados.
Configura o modelo para fine-tuning eficiente usando LoRA.
Define o template de prompt para formatar os dados para o treinamento.
Carrega o dataset de treinamento customizado e o formata no template do prompt.
Configura o treinador (SFTTrainer) com hiperparâmetros para o fine-tuning.
Executa o treinamento do modelo nos dados customizados.
Salva o modelo e tokenizador fine-tuned.
Demonstra como carregar o modelo salvo para inferência.
Realiza uma inferência com o modelo treinado para testar sua capacidade de gerar respostas com base em um título.
Observações Importantes:

O uso do Unsloth otimiza significativamente o processo de fine-tuning e o uso de memória em GPUs limitadas como as do Google Colab.
O pré-processamento dos dados customizados é crucial para garantir que o modelo aprenda a tarefa de forma eficaz.
A configuração dos parâmetros LoRA e dos argumentos de treinamento é fundamental para o desempenho e a estabilidade do fine-tuning.
O template de prompt utilizado (alpaca_prompt) define como as instruções, inputs e outputs são apresentados ao modelo durante o treinamento e a inferência. É importante que o formato da inferência corresponda ao formato de treinamento.
O template de chat "gemma-3" usado na Célula 6 pode ser inadequado para o modelo Llama 3.1. Verificar a documentação do Unsloth ou do modelo Llama 3.1 para o template de chat correto é recomendado.

In [ ]:
# Prepara a entrada para o modelo fine-tuned.
inputs = tokenizer(
[
    alpaca_prompt.format(
       "Answer the question based on the title and include the title in response.",  # instruction em inglês
       "Paris: City of Light",                   # pergunta diferente do padrão treinado
       "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

print (text_streamer)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the question based on the title and include the title in response.

### Input:
Paris: City of Light

### Response:
A fascinating, informative and well-written guide to the city of Paris. The author's enthusiasm for the city is contagious, and his knowledge of its history, architecture, and culture is impressive. This book is a great introduction to Paris.<|end_of_text|>


### **▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄▄**

## **PARTE 6: Exemplo dos arquivos utilizados**

# **Exemplificando arquivo: trn.json**

{"uid": "0000031909", "title": "Girls Ballet Tutu Neon Pink", "content": "High quality 3 layer ballet tutu. 12 inches in length", "target_ind": [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 111], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{"uid": "0000032034", "title": "Adult Ballet Tutu Yellow", "content": "", "target_ind": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16, 33, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{"uid": "0000913154", "title": "The Way Things Work: An Illustrated Encyclopedia of Technology", "content": "", "target_ind": [116, 117, 118, 119, 120, 121, 122], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{"uid": "0001360000", "title": "Mog's Kittens", "content": "Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.", "target_ind": [146, 147, 148, 149, 495], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0]}
{"uid": "0001381245", "title": "Misty of Chincoteague", "content": "", "target_ind": [151], "target_rel": [1.0]}
{"uid": "0001371045", "title": "Hilda Boswell's treasury of children's stories: A new anthology of stories for the young", "content": "", "target_ind": [150], "target_rel": [1.0]}
{"uid": "0000230022", "title": "The Simple Truths of Service: Inspired by Johnny the Bagger", "content": "", "target_ind": [184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{"uid": "0000031895", "title": "Girls Ballet Tutu Neon Blue", "content": "Dance tutu for girls ages 2-8 years. Perfect for dance practice, recitals and performances, costumes or just for fun!", "target_ind": [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 27, 31, 33, 42, 46, 54, 58, 111, 113, 125, 126, 159, 163, 202, 203, 204, 205, 206, 207], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{"uid": "0000174076", "title": "Evaluating Research in Academic Journals - A Practical Guide to Realistic Evaluation (5th Fifth Edition) - By Fred Pyrczak", "content": "", "target_ind": [106, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{"uid": "0001713086", "title": "Dr. Seuss ABC (Dr.Seuss Classic Collection) (Spanish Edition)", "content": "", "target_ind": [260, 261, 262, 263, 264, 265, 266, 267], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}

# **Exemplificando arquivo: tst.json**

{"uid": "0000032069", "title": "Adult Ballet Tutu Cheetah Pink", "content": "", "target_ind": [0, 1, 2, 4, 7, 8], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{"uid": "0000589012", "title": "Why Don't They Just Quit? DVD Roundtable Discussion: What Families and Friends need to Know About Addiction and Recovery", "content": "", "target_ind": [72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{"uid": "0000031852", "title": "Girls Ballet Tutu Zebra Hot Pink", "content": "TUtu", "target_ind": [13, 16, 18, 20, 23, 32, 33, 113, 115], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{"uid": "0000032050", "title": "Adult Ballet Tutu Purple", "content": "", "target_ind": [1, 2, 4, 7, 8, 31, 32, 35, 41, 46, 53, 60, 61, 123, 124, 131, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{"uid": "0001203088", "title": "Hilda Boswell's Omnibus - A Treasury of Favorites", "content": "", "target_ind": [150], "target_rel": [1.0]}
{"uid": "0000031887", "title": "Ballet Dress-Up Fairy Tutu", "content": "This adorable basic ballerina tutu is perfect for dance recitals. Fairy Princes Dress up, costume, play and much. Comes individually packaged. Use for a Tinkerbell dress up accessory and watch her flutter excitedly for hours in her tutu. Very soft elastic waist that is trimmed in satin and stretches to fit from an average size 3 to a size 8.", "target_ind": [3, 6, 9, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 30, 31, 32, 33, 35, 40, 42, 46, 53, 111, 112, 113, 114, 115, 126, 127, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]}
{"uid": "0001473727", "title": "The Greatest Book on &quot;Dispensational Truth&quot; in the World", "content": "", "target_ind": [181, 182], "target_rel": [1.0, 1.0]}
{"uid": "0001061127", "title": "Chess for Young Beginners", "content": "", "target_ind": [183], "target_rel": [1.0]}
{"uid": "0000013714", "title": "Heavenly Highway Hymns: Shaped-Note Hymnal", "content": "", "target_ind": [250, 257, 259], "target_rel": [1.0, 1.0, 1.0]}
{"uid": "0001381245", "title": "Misty of Chincoteague", "content": "", "target_ind": [151], "target_rel": [1.0]}

# **Exemplificando arquivo: formatted_train_dataset.json**

[
    {
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Manuka Honey and Australian Kakadu Plum Shampoo'?",
        "output": "Manuka Honey and Kakadu Plum Shampoo 250 ml/8.4 fl.oz. Our shampoo combines the protective and moisturizing properties of Manuka Honey with the high levels of natural Vitamin C of the Australian Kakadu Plum. Manuka Honey is known to protect the scalp and moisturize hair in a natural way. Kakadu Plum (Terminalia ferdinandiana) has the highest concentration of Vitamin C of any fruit (3200-5000mg/100g, compared with 50mg/100g for oranges). This high vitamin content has made the Kakadu Plum a mainstay of Australian Aboriginal practices. Vitamin C is a powerful antioxidant that serves to promote healthy hair. A new formula (released April 2012) includes proteins extracted from Jojoba leaves, which coat hair strands providing a protective film that in most cases obviates the use of conditioners."
    },
    {
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'ANTHEIL: Symphonies Nos. 4 and 6 / McKonkey's Ferry'?",
        "output": "ANTHEIL: Symphonies Nos. 4 and 6 / McKonkey's Ferry by Theodore KucharThis product is manufactured on demand using CD-R recordable media. Amazon.com's standard return policy will apply."
    },
    {
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Scrub Free Bathroom Cleaner with Oxi Clean, Lemon Scent, 32 oz'?",
        "output": "Scrub Free Soap Scum Remover With Oxy Cuts Through Soap Scum On Contact. Cleans And Deodorizes. No Scrubbing, No Scratching, Premium Performance, Premium Value"
    },
    {
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'White Sierra Women's Lakeport Skimmer Short, Medium, Bark'?",
        "output": "The Lakeport Skimmer is versatile for a walk in the park or a day at the beach. Relaxed fit and quick dry fabric will keep you comfortable all day. Sun protection and wicking functions are perfect for summertime fun."
    }
]    

# **Exemplificando arquivo: formatted_test_dataset.json**

[
    {
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Tank Killing: Anti-tank Warfare By Men And Machines'?",
        "output": "\"An intriguing study of the history of anti-tank defenses from WWI to the present.\" --Stars and Stripes\"The first comprehensive record of tactical and strategic antitank warfare, Tank Killing contains a wealth of data and insight for everyone concerned with modern warfare . . . a solid and rewarding study.\" --ArmyTank Killing address the subject of anti-tank warfare. With its debut in the mud fields of the First World War's &quot;Western Front&quot;, the tank has been recognized as a decisive component in land warfare, and so soldiers and scientists have used all their ingenuity to devise ways to destroy it. Tank Killing is the first record of the history of anti-tank weapons, tactics and defenses up to the present day. Tank Killing features the accounts of infantrymen on a variety of fronts and the wide array of ad hoc methods they employed to destroy the oncoming monsters. Sophisticated anti-tank weapons and tactics are thoroughly surveyed along with information on anti-tank helicopters and fixed-wing aircraft. Tank Killing is a vital addition to any military science curriculum and military studies reading list. --Midwest Book Review"
    },
    {
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'Raybestos FRC11268 Professional Grade Remanufactured, Semi-Loaded Disc Brake Caliper'?",
        "output": "Raybestos Friction Ready Caliper is designed to maximize performance and increase durability. This caliper features a corrosion resistant high-temperature synthetic lubricant and improves trouble-free operation. It improves braking and ensures safe, leak-free performance."
    },
    {
        "instruction": "Answer the question based on the product title.",
        "input": "What is 'HappyFeet Two Freezing Frenzy'?",
        "output": "Happy Feet Two Freezing Frenzy includes Erik, game base, 24 place ice cubes, and rules. (requires three AAA batteries - not included). It's the frantic action game where Erik sends ice cubes popping out all over, turn him on and he jumps and moves around the game base, kicking out colorful ice cubes along the way. Quickly grab your color cubes and quickly try to put the other players' cubes back in the base before they can grab them. The first player to get all of their cubes wins. For 2 to 4 players. Ages 5 and up."
    }
]